# EmissãoNF

Esse mini projeto de emissor de notas fiscais foi criado para mostrar como seria cansativo e tedioso realizar esse trabalho repetitivo manualmente todos os dias. Com a ajuda de um robô, esse processo pode ser automatizado, trazendo muita praticidade e eficiência.

No projeto, demonstre todas as etapas da emissão de notas fiscais, desde o login até o preenchimento dos dados necessários. O projeto é escalonável e capaz de emitir notas infinitas, desde que as informações estejam disponíveis no banco de dados.

In [ ]:
# Importações para o funcionamento
import os
import pandas as pd
import time
from credenciais import *

# Importações para controlar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# CONFIGURAR O NAVEGADOR PARA DOWNLOAD
# Configurações para permitir o download automático de arquivos
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
  "download.default_directory": r"C:\Users\carlo\Documents\GitHub\PastaPublica\Automatizar Emissão de Nota Fiscal\download",  # Diretório padrão para downloads
  "download.prompt_for_download": False,  # Desabilita o prompt de confirmação de download
  "download.directory_upgrade": True,  # Permite atualização do diretório
  "safebrowsing.enabled": True  # Habilita a navegação segura
})

# Configuração do serviço do WebDriver para o Chrome
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico, options=options)

# ENTRANDO NA PÁGINA DE LOGIN
# Carrega a página de login local
caminho = os.getcwd()
local = caminho + r'\login.html'
navegador.get(local)

navegador.maximize_window()  # Maximiza a janela do navegador

time.sleep(1)  # Pequena pausa para evitar problemas de timing
# Preenche os campos de login
navegador.find_element(By.ID, 'login').send_keys(login, Keys.TAB)  # Preenche o usuário

time.sleep(1)  # Pequena pausa para evitar problemas de timing
navegador.find_element(By.ID, 'senha').send_keys(senha, Keys.TAB, Keys.ENTER)  # Preenche a senha e faz login

# DADOS DA NOTA
# Lê os dados das notas fiscais a serem emitidas a partir de um arquivo Excel
dados = pd.read_excel('./NotasEmitir.xlsx')

# Obtém os títulos dos campos na página
titulo_site = [titulo.text for titulo in navegador.find_elements(By.TAG_NAME, 'label')]

# Cria um dicionário para mapear os títulos aos valores do Excel
dicionario = {
    titulo_site[0]: dados['Cliente'].values,
    titulo_site[1]: dados['Endereço'].values,
    titulo_site[2]: dados['Bairro'].values,
    titulo_site[3]: dados['Municipio'].values,
    titulo_site[4]: dados['CEP'].values,
    titulo_site[5]: dados['UF'].values,
    titulo_site[6]: dados['CPF/CNPJ'].values,
    titulo_site[7]: dados['Inscricao Estadual'].values,
    titulo_site[8]: dados['Descrição'].values,
    titulo_site[9]: dados['Quantidade'].values,
    titulo_site[10]: dados['Valor Unitario'].values,
    titulo_site[11]: dados['Valor Total'].values
}

# Cria uma tabela com os valores a serem inseridos
tabelaValores = pd.DataFrame.from_dict(dicionario)

# BAIXAR UMA NOTA POR VEZ
# Obtém os nomes dos campos na página e ajusta a posição do campo "uf"
nome = [titulo.get_attribute('name') for titulo in navegador.find_elements(By.TAG_NAME, 'input')]
cep_id = nome.index('cep')
nome.insert(cep_id + 1, 'uf')

# Preenche os dados de cada nota e realiza o download
for lista in tabelaValores.values:
    for id, valor in enumerate(lista):
        navegador.find_element(By.NAME, nome[id]).send_keys(valor, Keys.TAB)  # Preenche os campos com os valores correspondentes
        time.sleep(0.5)  # Pequena pausa para evitar problemas de timing

    # Clica no botão para baixar a nota
    navegador.find_element(By.XPATH, '//*[@id="formulario"]/button').click()

    # Atualiza a página para limpar os campos
    navegador.refresh()

# Fecha o navegador
navegador.quit()
